In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer

In [6]:
df = pd.read_csv('titanic.csv',usecols=['Age','Fare','SibSp','Parch','Survived'])

In [7]:
df.dropna(inplace=True)

In [8]:
df.head()

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500


In [9]:
df['family'] = df['SibSp'] + df['Parch']

In [10]:
df.head()

,Survived,Age,SibSp,Parch,Fare,family
0,0,22.0,1,0,7.2500,1
1,1,38.0,1,0,71.2833,1
2,1,26.0,0,0,7.9250,0
3,1,35.0,1,0,53.1000,1
4,0,35.0,0,0,8.0500,0


In [12]:
df.drop(columns=['SibSp','Parch'],inplace=True)

In [13]:
df.head()

,Survived,Age,Fare,family
0,0,22.0,7.2500,1
1,1,38.0,71.2833,1
2,1,26.0,7.9250,0
3,1,35.0,53.1000,1
4,0,35.0,8.0500,0


In [15]:
X=df.drop(columns=['Survived'])
Y=df['Survived']

In [16]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [17]:
X_train.head()

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0


In [19]:
#without binarization

clf = DecisionTreeClassifier()

clf.fit(X_train,Y_train)

Y_pred = clf.predict(X_test)

accuracy_score(Y_pred,Y_test)

0.6433566433566433

In [20]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,Y,cv=10,scoring='accuracy'))

0.6457159624413146

In [25]:
#applying binarization

from sklearn.preprocessing import Binarizer

In [26]:
#transforming family column into column travelling alone
trf = ColumnTransformer([
    ('bin',Binarizer(copy=False),['family'])
],remainder='passthrough')

In [27]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [29]:
pd.DataFrame(X_train_trf,columns=['family','Age','fare'])

,family,Age,fare
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [30]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,Y_train)
Y_pred2 = clf.predict(X_test_trf)

accuracy_score(Y_test,Y_pred2)

0.6153846153846154

In [31]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,Y,cv=10,scoring='accuracy'))

0.6219874804381847